In [1]:
import pandas as pd
import re

In [2]:
# === 0. Đọc dữ liệu ===
df = pd.read_csv('CellphoneS.csv', encoding='utf-8')

In [3]:
# === 1. Xóa các cột không cần thiết nếu có ===
columns_to_drop = [
    'web-scraper-order', 'web-scraper-start-url',
    'link', 'link-href', 'type'
]
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns], errors='ignore')

In [4]:
# === 2. Xử lý cột giá (price) ===
df['price'] = df['price'].astype(str).str.replace(r'[^\d]', '', regex=True)
df['price'] = pd.to_numeric(df['price'], errors='coerce')

In [5]:
# === 3. Hàm trích số từ chuỗi ===
def extract_number(x):
    if pd.isnull(x): return None
    x = str(x).replace(',', '')
    match = re.search(r'(\d+)', x)
    return int(match.group(1)) if match else None

In [6]:
# === 4. Áp dụng cho internal_storage, battery ===
for col in ['internal_storage', 'battery']:
    if col in df.columns:
        df[col] = df[col].apply(extract_number)

In [7]:
# === 5. Xử lý RAM ===
def extract_ram_in_gb(ram_str):
    if pd.isnull(ram_str):
        return None
    
    # Làm sạch chuỗi: loại bỏ dấu phẩy, khoảng trắng thừa, chuyển thành chữ thường
    cleaned_ram_str = str(ram_str).replace(',', '').strip().lower()

    # 1. Mẫu tìm kiếm để tính tổng RAM (có mở rộng) Ví dụ: "4GB + Mở rộng 4GB" sẽ bắt 4 và 4, tổng là 8.
    match_total = re.search(r'(\d+)\s*gb(?:\s*\+\s*mở\s*rộng\s*(\d+)\s*gb)?', cleaned_ram_str)
    if match_total:
        base_ram = int(match_total.group(1))
        expanded_ram = int(match_total.group(2)) if match_total.group(2) else 0
        return float(base_ram + expanded_ram) # Trả về tổng dưới dạng float (ví dụ: 8.0)

    # 2. Xử lý các trường hợp đơn giản chỉ có 'X GB' hoặc 'X MB'. Ví dụ: "4 GB" sẽ bắt 4 và 'gb'. "64MB" sẽ bắt 64 và 'mb'.
    match_simple = re.search(r'(\d+)\s*(gb|mb)?', cleaned_ram_str)
    if match_simple:
        num = int(match_simple.group(1))
        unit = match_simple.group(2)      
        if unit == 'mb':
            return float(num / 1024) # Chuyển MB sang GB
        else: # Mặc định là GB nếu không có đơn vị hoặc là 'gb'
            return float(num) # Trả về số nguyên dưới dạng float (ví dụ: 4.0)        
    return None 

df = pd.DataFrame(df)
# Áp dụng hàm chỉ cho cột 'RAM'
df['RAM_in_GB'] = df['RAM'].apply(extract_ram_in_gb)

print(df[['RAM', 'RAM_in_GB']].head(20)) # In 20 hàng đầu để thấy kết quả

                  RAM  RAM_in_GB
0               12 GB       12.0
1               12 GB       12.0
2   4GB + Mở rộng 4GB        8.0
3   8GB + Mở rộng 8GB       16.0
4                6 GB        6.0
5                 NaN        NaN
6                8 GB        8.0
7   6GB + Mở rộng 6GB       12.0
8   4GB + Mở rộng 4GB        8.0
9                4 GB        4.0
10                NaN        NaN
11               8 GB        8.0
12               8 GB        8.0
13               6 GB        6.0
14               4 GB        4.0
15              16 GB       16.0
16              12 GB       12.0
17              12 GB       12.0
18               8 GB        8.0
19              16 GB       16.0


In [8]:
# Xóa cột RAM 
df.drop('RAM', axis=1, inplace=True)

In [9]:
# === 6. Xử lý ratings: '4.5/5' → 4.5
if 'ratings' in df.columns:
    df['ratings'] = df['ratings'].astype(str).str.extract(r'([\d.]+)')
    df['ratings'] = pd.to_numeric(df['ratings'], errors='coerce')

In [11]:
# === 7. Xử lý refresh_rate: trích số Hz từ chuỗi
def extract_refresh_rate(x):
    match = re.search(r'(\d{2,3})\s*Hz', str(x))
    if match:
        return int(match.group(1))
    return None
if 'refresh_rate' in df.columns:
    df['refresh_rate'] = df['refresh_rate'].apply(extract_refresh_rate)

In [12]:
# === 8. Xử lý screen_size: trích kích thước inch từ chuỗi
def extract_screen_size(x):
    match = re.search(r'([\d.]+)\s*(inch|\"|\')', str(x), flags=re.IGNORECASE)
    if match:
        return float(match.group(1))
    return None
if 'screen_size' in df.columns:
    df['screen_size'] = df['screen_size'].apply(extract_screen_size)

In [13]:
df.head()

,title,price,ratings,num_rates,screen_size,refresh_rate,chipset,GPU,sim_slot,internal_storage,battery,operating_system,RAM_in_GB
0,ASUS ROG Phone 6 Mediatek,NaN,NaN,NaN,6.78,None,MediaTek Dimensity 7000 Series,Adreno ™ 730,2 SIM (Nano-SIM),256.0,6000.0,Android 12,12.0
1,ASUS ROG Phone 6 12GB 256GB,14490000.0,5.0,20.0,6.78,None,Qualcomm ® Snapdragon ® 8+ thế hệ 1,Adreno ™ 730,2 SIM (Nano-SIM),256.0,6000.0,Android 12,12.0
2,BENCO V91 4GB 128GB,2690000.0,5.0,1.0,6.56,None,Unisoc T606,NaN,2 SIM (Nano-SIM),128.0,5000.0,Android 13,8.0
3,Benco S1 Pro 8GB 256GB,4290000.0,NaN,NaN,6.80,None,Unisoc T616,Mali-G57,2 SIM (Nano-SIM),256.0,5000.0,Android 13,16.0
4,INOI A83 6GB 128GB,NaN,NaN,NaN,6.52,None,Unisoc T606,NaN,2 SIM (Nano-SIM),128.0,4000.0,Android 12,6.0


In [13]:
print(df.isnull().sum())


title                 0
price               105
ratings             139
num_rates           139
screen_size           7
refresh_rate        103
chipset              14
GPU                 101
sim_slot             38
internal_storage     15
battery              19
operating_system     45
RAM_in_GB            37
dtype: int64


In [14]:
# === 9. Loại bỏ dòng không có giá
df = df.dropna(subset=['price'])

In [15]:
# === 10. Điền median cho các cột số ===
numeric_cols = ['num_rates', 'RAM_in_GB', 'internal_storage', 'battery', 'refresh_rate']
for col in numeric_cols:
    if col in df.columns:
        median_val = df[col].median()
        df[col] = df[col].fillna(median_val)

In [16]:
# === 11. Điền mode cho các cột phân loại ===
categorical_cols = ['ratings', 'screen_size', 'chipset', 'GPU', 'sim_slot', 'operating_system']
for col in categorical_cols:
    if col in df.columns:
        if df[col].isnull().any():
            df[col] = df[col].fillna(df[col].mode()[0])

In [17]:
def simplify_os(os_name):
    os_name = os_name.lower()
    if "android" in os_name:
        return "Android"
    elif "ios" in os_name:
        return "iOS"
    elif "s30" in os_name:
        return "S30+"
    else:
        return "Khác"

df['operating_system'] = df['operating_system'].apply(simplify_os)


In [18]:
# === 12. Xuất file Excel
#df.to_csv('du_lieu_da_xu_ly.csv', index=False,encoding='utf-8-sig')

In [19]:
# === 13. In kiểm tra
print("✅ Dữ liệu đã xử lý xong và lưu vào 'du_lieu_da_xu_ly.xlsx'")
print(df.head())
print("\n⚠️ Kiểm tra thiếu giá trị:")
print(df.isnull().sum())

✅ Dữ liệu đã xử lý xong và lưu vào 'du_lieu_da_xu_ly.xlsx'
                         title       price  ratings  num_rates  screen_size  \
1  ASUS ROG Phone 6 12GB 256GB  14490000.0      5.0       20.0         6.78   
2          BENCO V91 4GB 128GB   2690000.0      5.0        1.0         6.56   
3       Benco S1 Pro 8GB 256GB   4290000.0      5.0       10.0         6.80   
5                 INOI 288S 4G   1000000.0      5.0        1.0         2.00   
6      INOI Note 13s 8GB 256GB   2790000.0      5.0        2.0         6.95   

   refresh_rate                              chipset           GPU  \
1         120.0  Qualcomm ® Snapdragon ® 8+ thế hệ 1  Adreno ™ 730   
2          90.0                          Unisoc T606    Adreno 610   
3         120.0                          Unisoc T616      Mali-G57   
5         120.0                        Apple A18 Pro    Adreno 610   
6         120.0                          Unisoc T606    Adreno 610   

           sim_slot  internal_storage  batter